In [1]:
import seaborn as sns
iris = sns.load_dataset("iris")
iris.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


In [2]:
#Split the data
from sklearn import svm
from sklearn.model_selection import train_test_split

#classes = iris.target_names.tolist()

y = iris.species #Label
X = iris.drop('species',axis=1) #Features

# split data into train and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,            
                                                    y,  
                                                    test_size=0.2,
                                                    random_state=0,
                                                    stratify=y)


In [3]:
#Name the classes
classes = ['setosa', 'versicolor', 'virginica']

#Create a function to convert species to numeric
def func(x):
     if x == 'setosa':
          return 1
     elif x == 'versicolor':
          return 2
     elif x == 'virginica':
          return 3

from numpy import vectorize
vfunc = vectorize(func) #Define a vectorized function which takes a nested sequence of objects or numpy arrays as inputs

y_train = vfunc(y_train)
y_test = vfunc(y_test)


In [4]:
%%time
#Train a tree
from sklearn import tree
from sklearn.metrics import confusion_matrix

clf = tree.DecisionTreeClassifier(criterion="gini", random_state=1)
model = clf.fit(x_train,y_train)
y_pred = (model.predict(x_test))

cm = confusion_matrix(y_test, y_pred)
print(cm)

[[10  0  0]
 [ 0 10  0]
 [ 0  1  9]]
CPU times: user 10.3 ms, sys: 439 µs, total: 10.7 ms
Wall time: 10.4 ms


In [5]:
from interpret.ext.blackbox import TabularExplainer

# "features" and "classes" fields are optional
explainer = TabularExplainer(model, 
                             x_train, 
                             classes=classes)

In [6]:
#Run the explainer
#import warnings
#We get a lot of warning so filter them out for this
#warnings.filterwarnings('ignore')

global_explanation = explainer.explain_global(x_test)

In [7]:
#Get the global feature importance
global_explanation.get_feature_importance_dict()


{'petal_width': 0.3945484854110586,
 'petal_length': 0.07408467155543179,
 'sepal_length': 0.01817089018843405,
 'sepal_width': 0.0}

In [8]:
import pandas as pd
pd.DataFrame(global_explanation.get_ranked_global_values(), global_explanation.get_ranked_global_names())


,0
petal_width,0.394548
petal_length,0.074085
sepal_length,0.018171
sepal_width,0.000000


In [10]:
##Warning - Additional setup needed to run in Jupyter labs
from interpret_community.widget import ExplanationDashboard

ExplanationDashboard(global_explanation, model, datasetX=x_test)

ExplanationWidget(value={'predictedY': [1, 2, 1, 3, 1, 2, 3, 1, 1, 2, 3, 2, 2, 3, 2, 3, 3, 2, 2, 1, 1, 3, 3, 2…